Aqui montaremos a rede para avaliar os dados para a tarefa de classificação multiclasse com a extração de características hog (Histogram of Oriented Gradients)

Importar as bibliotecas e os dados do conjunto MNIST

In [24]:
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler

# 1. Carregar o conjunto de dados MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Criando a função que implementa o descritor HOG

In [25]:
def extract_hog_features(images):
    features = []
    for image in images:
        fd = hog(image, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(1, 1), visualize=False)
        features.append(fd)
    return np.array(features)

Normalizando os dados. As imagens já vem em escala de cinza.

In [26]:
x_train = x_train / 255.0
x_test = x_test / 255.0

Extraír as características HOG do conjunto

In [27]:
x_train_hog = extract_hog_features(x_train)
x_test_hog = extract_hog_features(x_test)

Normalizando as características HOG extraídas

In [28]:
scaler = StandardScaler()
x_train_hog = scaler.fit_transform(x_train_hog)
x_test_hog = scaler.transform(x_test_hog)

Redimensionando os dados normalizados, extraídos com HOG, para o formato esperado pela CNN

In [29]:
x_train_hog = x_train_hog.reshape(x_train_hog.shape[0], 1, 1, x_train_hog.shape[1])
x_test_hog = x_test_hog.reshape(x_test_hog.shape[0], 1, 1, x_test_hog.shape[1])

Converter os rótulos para one-hot encoding

`to_categorical(rótulos, número de classes)`

In [30]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

Construir o modelo

In [31]:
model = Sequential([
    Conv2D(32, kernel_size=(1, 1), activation='relu', input_shape=(1, 1, x_train_hog.shape[3])),
    MaxPooling2D(pool_size=(1, 1)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

c:\Users\mtoma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Compilar o modelo

In [32]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

- Criando e salvando os hiperparâmetros da arquitetura e da inicialização em formato json em arquivos-hog/hiperparametros.json

- Salvando os pesos iniciais

In [33]:
# Definindo os hiperparâmetros
hiperparametros = {
    "arquitetura": {
        "camadas": [64, 64, 10],
        "ativacao": "relu",
        "ultimo_ativacao": "softmax"
    },
    "inicializacao": {
        "pesos": "he_normal",
        "bias": "zeros"
    }
}

# Serializando os hiperparâmetros em uma string JSON
hiperparametros_json = json.dumps(hiperparametros, indent=4)

# Escrevendo a string JSON em um arquivo
with open("arquivos-hog/hiperparametros.json", "w") as arquivo:
    arquivo.write(hiperparametros_json)


# PESOS INICIAIS
model.save_weights('arquivos-hog/pesos_iniciais.weights.h5')

Treinar o modelo

In [34]:
historico = model.fit(x_train_hog, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8546 - loss: 0.4679 - val_accuracy: 0.9676 - val_loss: 0.1009
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9756 - loss: 0.0754 - val_accuracy: 0.9704 - val_loss: 0.0927
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9836 - loss: 0.0521 - val_accuracy: 0.9718 - val_loss: 0.0934
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9889 - loss: 0.0360 - val_accuracy: 0.9722 - val_loss: 0.1007
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9910 - loss: 0.0286 - val_accuracy: 0.9725 - val_loss: 0.1070
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9924 - loss: 0.0228 - val_accuracy: 0.9718 - val_loss: 0.1182
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9931 - loss: 0.0196 - val_accuracy: 0.9725 - val_loss: 0.1279
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9933 - loss: 0.0178 - 

- Salvando os pesos finais da rede 
- Salvando o histórico de perda para cada iteração
- Salvando as saídas produzidas pela rede para cada um dos dados de teste

In [35]:
# PESOS FINAIS
model.save_weights('arquivos-hog/pesos_finais.weights.h5')

# ERRO DE CADA ITERAÇÃO
perdas = historico.history['loss']

# Salvando o histórico de perda em um arquivo JSON
with open('arquivos-hog/historico_perda.json', 'w') as f:
    json.dump(perdas, f)


# SAÍDAS PRODUZIDAS
# Fazendo inferência com o modelo treinado para obter as saídas
saidas = model.predict(x_train_hog)

# Convertendo as saídas para uma lista para serialização
saidas_lista = saidas.tolist()

# Salvando as saídas em um arquivo JSON
with open('arquivos-hog/saidas_teste.json', 'w') as f:
    json.dump(saidas_lista, f)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


Testar o modelo

In [36]:
test_loss, test_acc = model.evaluate(x_test_hog, y_test)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9681 - loss: 0.1518
Test accuracy: 0.9746999740600586, Test loss: 0.12003465741872787
